## Level 2 - Word Prediction using LSTM

Follow the same steps as in Char Prediction (Level 1) but at the word level than at the Char Level. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import cross_validation
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot    
import plotly.graph_objs as go
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from sklearn.cross_validation import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize,regexp_tokenize

/home/user/classification/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
text = """ CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, and what is the use of a book, thought Alice without pictures or
conversations?

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure
of making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.
CHAPTER II. The Pool of Tears

Curiouser and curiouser! cried Alice (she was so much surprised, that
for the moment she quite forgot how to speak good English); now I am
opening out like the largest telescope that ever was! Good-bye, feet!
(for when she looked down at her feet, they seemed to be almost out of
sight, they were getting so far off). Oh, my poor little feet, I wonder
who will put on your shoes and stockings for you now, dears? I am sure
I shall not be able! I shall be a great deal too far off to trouble
myself about you, you must manage the best way you can;but I must be
kind to them, thought Alice, or perhaps they wont walk the way I want
to go! Let me see: I will give them a new pair of boots every Christmas.


*    *    *    *    *    *    * """

In [3]:
def regex_token(list):
    words = []    
    words_2 = regexp_tokenize(list, pattern=r'\w+')
    words.append(words_2)
    return words_2
   

In [39]:
words2 = regex_token(text)
len(words2)

267

In [12]:
words = set(words2)
len(words)

162

In [6]:
word_to_number = {word:i for i,word in enumerate(words2)}

count = 0
for word,i in word_to_number.items():
    word_to_number[word] = count
    count+=1
    
int_to_word = {i:word for word,i in word_to_number.items()}

In [9]:
max(word_to_number.values()) 

161

In [13]:
word_to_number    

{'Alice': 147,
 'CHAPTER': 108,
 'Christmas': 134,
 'Curiouser': 72,
 'Down': 141,
 'English': 74,
 'Good': 78,
 'Hole': 99,
 'I': 7,
 'II': 154,
 'Let': 4,
 'Oh': 138,
 'Pool': 53,
 'Rabbit': 18,
 'So': 31,
 'Tears': 59,
 'The': 158,
 'White': 90,
 'a': 52,
 'able': 105,
 'about': 46,
 'almost': 113,
 'am': 12,
 'and': 21,
 'as': 20,
 'at': 87,
 'bank': 73,
 'be': 121,
 'beginning': 161,
 'best': 70,
 'book': 119,
 'boots': 39,
 'but': 112,
 'by': 122,
 'bye': 13,
 'can': 26,
 'chain': 43,
 'close': 120,
 'considering': 14,
 'conversations': 64,
 'could': 117,
 'cried': 130,
 'curiouser': 98,
 'daisies': 49,
 'daisy': 131,
 'day': 45,
 'deal': 32,
 'dears': 104,
 'do': 23,
 'down': 160,
 'ever': 24,
 'every': 144,
 'eyes': 151,
 'far': 159,
 'feel': 101,
 'feet': 132,
 'for': 50,
 'forgot': 33,
 'get': 123,
 'getting': 56,
 'give': 11,
 'go': 19,
 'good': 106,
 'great': 66,
 'had': 85,
 'having': 97,
 'her': 15,
 'hot': 16,
 'how': 148,
 'in': 25,
 'into': 69,
 'is': 47,
 'it': 91,
 '

In [21]:
text_list = []
for i in words2:
    text_list.append(word_to_number[i])
text_list    

[108,
 7,
 141,
 8,
 18,
 99,
 147,
 34,
 161,
 67,
 123,
 40,
 128,
 36,
 55,
 122,
 15,
 41,
 6,
 8,
 73,
 21,
 36,
 97,
 107,
 67,
 23,
 63,
 116,
 100,
 149,
 85,
 75,
 69,
 8,
 119,
 15,
 41,
 34,
 137,
 112,
 91,
 85,
 125,
 29,
 116,
 64,
 25,
 91,
 21,
 83,
 47,
 8,
 129,
 36,
 52,
 119,
 5,
 147,
 127,
 29,
 116,
 64,
 31,
 149,
 34,
 14,
 25,
 15,
 118,
 61,
 20,
 76,
 20,
 149,
 117,
 50,
 8,
 16,
 45,
 126,
 15,
 101,
 40,
 60,
 21,
 58,
 92,
 8,
 79,
 36,
 146,
 52,
 131,
 43,
 28,
 121,
 62,
 8,
 10,
 36,
 56,
 80,
 21,
 0,
 8,
 49,
 68,
 95,
 52,
 90,
 18,
 102,
 2,
 151,
 51,
 120,
 122,
 15,
 108,
 154,
 158,
 53,
 36,
 59,
 72,
 21,
 98,
 130,
 147,
 149,
 34,
 27,
 77,
 94,
 81,
 50,
 8,
 22,
 149,
 57,
 33,
 148,
 67,
 71,
 106,
 74,
 17,
 7,
 12,
 54,
 37,
 143,
 8,
 153,
 65,
 81,
 24,
 34,
 78,
 13,
 132,
 50,
 68,
 149,
 93,
 160,
 87,
 15,
 132,
 3,
 86,
 67,
 121,
 113,
 37,
 36,
 42,
 3,
 140,
 56,
 27,
 159,
 48,
 138,
 9,
 135,
 142,
 132,
 7,
 114,
 96,
 3

In [22]:
max(text_list)

161

In [23]:
def sliding_window(text,length):
    sqin = []
    sqout = []
    for i in range(len(text)):
        input = []
        output = []
        long = len(text)
        j = 0
        starting = i
        if((long-i)<(length+1)):
            break
        for j in range(length):
            input.append(text[starting+j])
        j+=1
        sqout.append(text[starting+j])
        sqin.append(input)
    #print(sqout)
    return sqin,sqout
            

In [24]:
sqin1,sqout = sliding_window(text_list,100)

In [26]:
sqin1

[[108,
  7,
  141,
  8,
  18,
  99,
  147,
  34,
  161,
  67,
  123,
  40,
  128,
  36,
  55,
  122,
  15,
  41,
  6,
  8,
  73,
  21,
  36,
  97,
  107,
  67,
  23,
  63,
  116,
  100,
  149,
  85,
  75,
  69,
  8,
  119,
  15,
  41,
  34,
  137,
  112,
  91,
  85,
  125,
  29,
  116,
  64,
  25,
  91,
  21,
  83,
  47,
  8,
  129,
  36,
  52,
  119,
  5,
  147,
  127,
  29,
  116,
  64,
  31,
  149,
  34,
  14,
  25,
  15,
  118,
  61,
  20,
  76,
  20,
  149,
  117,
  50,
  8,
  16,
  45,
  126,
  15,
  101,
  40,
  60,
  21,
  58,
  92,
  8,
  79,
  36,
  146,
  52,
  131,
  43,
  28,
  121,
  62,
  8,
  10],
 [7,
  141,
  8,
  18,
  99,
  147,
  34,
  161,
  67,
  123,
  40,
  128,
  36,
  55,
  122,
  15,
  41,
  6,
  8,
  73,
  21,
  36,
  97,
  107,
  67,
  23,
  63,
  116,
  100,
  149,
  85,
  75,
  69,
  8,
  119,
  15,
  41,
  34,
  137,
  112,
  91,
  85,
  125,
  29,
  116,
  64,
  25,
  91,
  21,
  83,
  47,
  8,
  129,
  36,
  52,
  119,
  5,
  147,
  127,
  29,
  116,


In [27]:
def reshaping(sqin):
    p = []
    for k in range(len(sqin)):
        q = []
        for i in range(len(sqin[0])):
            #print([sqin[k][i]])
            q.append([sqin[k][i]])
        p.append(q)
    return p

In [28]:
sqin = reshaping(sqin1)
sqin

[[[108],
  [7],
  [141],
  [8],
  [18],
  [99],
  [147],
  [34],
  [161],
  [67],
  [123],
  [40],
  [128],
  [36],
  [55],
  [122],
  [15],
  [41],
  [6],
  [8],
  [73],
  [21],
  [36],
  [97],
  [107],
  [67],
  [23],
  [63],
  [116],
  [100],
  [149],
  [85],
  [75],
  [69],
  [8],
  [119],
  [15],
  [41],
  [34],
  [137],
  [112],
  [91],
  [85],
  [125],
  [29],
  [116],
  [64],
  [25],
  [91],
  [21],
  [83],
  [47],
  [8],
  [129],
  [36],
  [52],
  [119],
  [5],
  [147],
  [127],
  [29],
  [116],
  [64],
  [31],
  [149],
  [34],
  [14],
  [25],
  [15],
  [118],
  [61],
  [20],
  [76],
  [20],
  [149],
  [117],
  [50],
  [8],
  [16],
  [45],
  [126],
  [15],
  [101],
  [40],
  [60],
  [21],
  [58],
  [92],
  [8],
  [79],
  [36],
  [146],
  [52],
  [131],
  [43],
  [28],
  [121],
  [62],
  [8],
  [10]],
 [[7],
  [141],
  [8],
  [18],
  [99],
  [147],
  [34],
  [161],
  [67],
  [123],
  [40],
  [128],
  [36],
  [55],
  [122],
  [15],
  [41],
  [6],
  [8],
  [73],
  [21],
  [36],
 

In [29]:
def dataframe(sqin,sqout):
    para = pd.DataFrame(
        {'sqin': sqin,
         'sqout': sqout
        })
    return para

In [30]:
para = dataframe(sqin,sqout) 


In [31]:
x_train, x_test = train_test_split(para, train_size = 0.8)
y_train = pd.DataFrame(x_train['sqout'])
y_test = pd.DataFrame(x_test['sqout'])
x_train.drop(['sqout'],axis=1,inplace=True)
x_test.drop(['sqout'],axis=1,inplace=True)

/home/user/classification/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/user/classification/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [33]:
from keras.utils import np_utils
np_utils.to_categorical?
ytrain = np_utils.to_categorical(y_train, num_classes=(max(text_list)))
ytest = np_utils.to_categorical(y_test, num_classes=(max(text_list)))

In [34]:
xtrain = np.array(x_train)
xtest = np.array(x_test)


In [35]:
def fun(xtrain):
    #xtrain = np.array(x_train)
    spd = []
    for i in range(len(xtrain)):
        spd.append(xtrain[i][0])
    spd =  np.array(spd)   
    return spd

In [36]:
xtrain = fun(xtrain)
xtest = fun(xtest)

In [37]:
xtrain.shape

(133, 100, 1)

In [38]:
model = Sequential()
model.add(LSTM(256, input_shape=(xtrain.shape[1],xtrain.shape[2])))
model.add(Dense(500, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(ytrain.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),metrics = ['accuracy'])
model.fit(xtrain, ytrain, epochs=5, batch_size=1, verbose=2,validation_split = .2)

Train on 106 samples, validate on 27 samples
Epoch 1/5
8s - loss: 5.1564 - acc: 0.0472 - val_loss: 5.2435 - val_acc: 0.0000e+00
Epoch 2/5
8s - loss: 4.7026 - acc: 0.0660 - val_loss: 5.4178 - val_acc: 0.0000e+00
Epoch 3/5
8s - loss: 4.3852 - acc: 0.0660 - val_loss: 5.8255 - val_acc: 0.0370
Epoch 4/5
8s - loss: 4.2189 - acc: 0.0755 - val_loss: 6.3464 - val_acc: 0.0000e+00
Epoch 5/5
8s - loss: 4.0884 - acc: 0.0755 - val_loss: 6.5375 - val_acc: 0.0370


word predict

In [40]:
predict = """So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure
of making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.
CHAPTER II. The Pool of Tears

Curiouser and curiouser! cried Alice (she was so much surprised, that
for the moment she quite forgot how to speak good English); now I am
opening out like the largest telescope that ever was! Good-bye, feet!
(for when she looked down at her feet, they seemed to be almost out of
sight, they were getting so far off). Oh, my poor little feet, I wonder
who will put on your shoes and stockings for you now, dears? I am sure
I shall not be able! I shall be a great deal too far off to trouble"""

In [41]:
predict_words = regex_token(predict)
len(predict_words)

159

In [43]:
text_list_predict = []
for i in predict_words:
    text_list_predict.append(word_to_number[i])
text_list_predict    

[31,
 149,
 34,
 14,
 25,
 15,
 118,
 61,
 20,
 76,
 20,
 149,
 117,
 50,
 8,
 16,
 45,
 126,
 15,
 101,
 40,
 60,
 21,
 58,
 92,
 8,
 79,
 36,
 146,
 52,
 131,
 43,
 28,
 121,
 62,
 8,
 10,
 36,
 56,
 80,
 21,
 0,
 8,
 49,
 68,
 95,
 52,
 90,
 18,
 102,
 2,
 151,
 51,
 120,
 122,
 15,
 108,
 154,
 158,
 53,
 36,
 59,
 72,
 21,
 98,
 130,
 147,
 149,
 34,
 27,
 77,
 94,
 81,
 50,
 8,
 22,
 149,
 57,
 33,
 148,
 67,
 71,
 106,
 74,
 17,
 7,
 12,
 54,
 37,
 143,
 8,
 153,
 65,
 81,
 24,
 34,
 78,
 13,
 132,
 50,
 68,
 149,
 93,
 160,
 87,
 15,
 132,
 3,
 86,
 67,
 121,
 113,
 37,
 36,
 42,
 3,
 140,
 56,
 27,
 159,
 48,
 138,
 9,
 135,
 142,
 132,
 7,
 114,
 96,
 35,
 84,
 6,
 110,
 82,
 21,
 133,
 50,
 111,
 17,
 104,
 7,
 12,
 157,
 7,
 1,
 115,
 121,
 105,
 7,
 1,
 121,
 52,
 66,
 32,
 109,
 159,
 48,
 67,
 10]

In [44]:
pred_slide,pred2 = sliding_window(text_list_predict,100)


In [45]:
pred_reshape = reshaping(pred_slide)


In [46]:
ar = np.array(pred_reshape)
ar.shape

(59, 100, 1)

In [47]:
result = model.predict(ar, batch_size=1, verbose=2)

In [50]:
result

array([[ 0.01198946,  0.00083588,  0.00521061, ...,  0.00606993,
         0.01282534,  0.00748976],
       [ 0.00527613,  0.00125739,  0.01079186, ...,  0.01110848,
         0.01459957,  0.01290777],
       [ 0.00107621,  0.0001815 ,  0.01130042, ...,  0.01335926,
         0.01461291,  0.01152434],
       ..., 
       [ 0.0010219 ,  0.0001599 ,  0.01092645, ...,  0.01304216,
         0.01421451,  0.0113442 ],
       [ 0.01265762,  0.00080273,  0.00481578, ...,  0.00578734,
         0.01305655,  0.00698407],
       [ 0.00527581,  0.00124198,  0.01074693, ...,  0.01112088,
         0.01464801,  0.01293947]], dtype=float32)

In [52]:
for i in range(result.shape[0]):
    res = np.argmax(result[i])
    print(list(word_to_number.keys())[list(word_to_number.values()).index(res)])

the
I
I
I
I
I
of
I
of
I
I
I
I
of
the
I
of
I
the
of
I
the
I
of
I
I
I
of
I
I
of
I
of
I
I
of
I
the
I
of
I
of
be
I
of
be
I
I
I
of
of
I
the
I
of
I
I
the
I
